In [1]:

%pprint
import sys
import os.path as osp

executable_path = sys.executable; scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts')
py_folder = osp.abspath('../py'); ffmpeg_folder = r'C:\ffmpeg\bin'
if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
from jobpostlib import (cu, datetime, duration, freq, hau, humanize, nu, scrfcu, slrcu, su, t0, time, wsu, speech_engine)
import os

Pretty printing has been turned OFF
======== Neo4j/5.24.2 ========
Utility libraries created in 7 seconds



----

In [8]:

wsu.save_email_to_file('indeed', verbose=True)

['powershell.exe', '-ExecutionPolicy', 'Bypass', '-Command', 'Start-Process "C:\\Program Files\\Notepad++\\notepad++.exe" -ArgumentList "C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\job-hunting\\data\\html\\indeed_email.html"']


In [9]:

# Get page soup
driver = wsu.get_driver(verbose=False); speech_engine.say('Browser available'); speech_engine.runAndWait()
viewjob_links_set = set(); location_strs_set = set()
file_path = '../data/html/indeed_email.html'
page_soup = wsu.get_page_soup(file_path)

In [10]:

# Get search type
search_type = 'Indeed jobs in 01510'
type_css = 'table > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table > tbody > tr > td > table > tbody > tr > td > table > tbody > tr:nth-child(1) > td > h1'
h1_list = page_soup.select(type_css)
if h1_list:
    import re
    search_type = re.sub(r"^\d+ new ", "Indeed ", h1_list[0].text).replace('job ', 'jobs ')
    print(search_type)

Indeed jobs in 01510


In [11]:

rows_css = 'table > tbody > tr'
cells_css = 'td > a > table > tbody > tr:nth-child(3) > td'
row_soups_list = page_soup.select(rows_css)
for row_soup in row_soups_list:
    cell_soups_list = row_soup.select(cells_css)
    if cell_soups_list:
        location_str = cell_soups_list[0].text.strip()
        if any(map(lambda x: x in location_str, ['Massachusetts', 'United States', 'Clinton, MA'])):
            location_strs_set.add(location_str)
            page_soup = list(cell_soups_list[0].parent.previous_siblings)[1]
            
            # Get viewjob links
            for link_soup in page_soup.select('a[href^="https://www.indeed.com/rc/clk/dl?jk="]'):
                new_url = su.clean_indeed_url(link_soup['href'], driver, verbose=False)
                viewjob_links_set.add(new_url)
            
            # Get pagead links and convert them to viewjob links
            for link_soup in page_soup.select('a[href^="https://www.indeed.com/pagead/clk"]'):
                new_url = su.clean_indeed_url(link_soup['href'], driver, verbose=False)
                viewjob_links_set.add(new_url)

speech_str = f'I found {len(viewjob_links_set)} urls (in {nu.conjunctify_nouns(location_strs_set)}) in indeed_email.html'; print(speech_str)

I found 10 urls (in Massachusetts and United States) in indeed_email.html


In [12]:

files_list = []; close_notices = True
for url_str in viewjob_links_set:
    file_node_dict, files_list = su.load_indeed_posting_url(
        viewjob_url=url_str, driver=driver, files_list=files_list, close_notices=close_notices, slowed_for_readability=False, search_type=search_type, verbose=False
    )
    close_notices = False
speech_str = f'Fileing {len(files_list)} postings out of {len(viewjob_links_set)} urls complete. Delete the email.'; print(speech_str)
speech_engine.say(speech_str); speech_engine.runAndWait()

Fileing 3 postings out of 10 urls complete. Delete the email.


In [13]:

su.populate_postings(driver, len(viewjob_links_set), files_list=files_list)

Populate the Child Strings: 100%|█████████████████| 3/3 [01:29<00:00, 29.74s/it]


['c4379d3fd26d548a_Freelance_Coding_Mentor_Massachusetts_Indeed_com.html', '8c59005d1465276b_Farsi_Linguist_and_Cultural_Advisor_Al_Udeid_AB_Qatar_TS_SCI_United_States_Indeed_com.html', '4801f7f2ece4891d_Software_Development_Coach_Massachusetts_Indeed_com.html']